# Libraries

In [0]:
import pandas as pd
import numpy as np
import csv
import json
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import plotly.express as px
from sklearn.feature_extraction.text import TfidfVectorizer as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# Read JSON and convert it to a dataframe


In [0]:
openfile=open('bulkV2.json')
jsondata=json.load(openfile)
df=pd.DataFrame(jsondata)
openfile.close()
#df.head(10)

# Create a subset that includes 3 columns: description,title,price

In [0]:
subset = df.loc[df['c1'] == 'Otomobil']
subset = subset[['c0','c1','c2','description','title','price']] 
#subset
del(df)

# Clean the dirty data

In [0]:
def preprocess(reviewText):
  reviewText = reviewText.str.replace("(<u>)", "")
  reviewText = reviewText.str.replace("(\t)", "")
  reviewText = reviewText.str.replace("(\n)", "")
  reviewText = reviewText.str.replace("<.*?>", "")
  return reviewText
  
subset['description'] = preprocess(subset['description'])
subset.index = range(0,len(subset))
#subset

In [0]:
#subset.to_csv (r'export_dataframe.csv', index = None, header=True) # this line saves the dataframe as .csv file

In [6]:
median = subset['price'].median()
lowerBound = subset.loc[subset['price']<=median]
firstQuartile = lowerBound['price'].median()
upperBound = subset.loc[subset['price']>=median]
thirdQuartile = upperBound['price'].median()
IQR = thirdQuartile-firstQuartile
minimumPrice = max(firstQuartile - 1.5*IQR,subset['price'].min())
maximumPrice = min(thirdQuartile + 1.5*IQR,subset['price'].max())
print(median,firstQuartile,thirdQuartile,IQR,minimumPrice,maximumPrice)

48500.0 30000.0 76500.0 46500.0 6000.0 146250.0


In [0]:
#datas in the interquartile range
normalDistOfDataframe = subset.loc[(subset['price']>=minimumPrice)]
normalDistOfDataframe = subset.loc[(subset['price']<=maximumPrice)]
normalDistOfDataframe.index = range(0,len(normalDistOfDataframe))
#normalDistOfDataframe

# Find most using 25 words in document (Vakit kalınca dönecektim word frequency'leri göstermek için fakat yetmedi.)



In [0]:

# def get_top_n_words(corpus,n=None):
#   vec = CountVectorizer().fit(corpus)
#   bag_of_words = vec.transform(corpus)
#   sum_words = bag_of_words.sum(axis=1)
#   word_freq = [(word, sum_words[0,idx]) for word,idx in vec.vocabulary_.items()]
#   word_freq = sorted(word_freq, key = lambda x:x[1], reverse=True)
#   return word_freq[:n]


# common_words = get_top_n_words(subset['title'],25)
# df_common_words = pd.DataFrame(common_words, columns=['word','frequency'])
# fig = px.bar(df_common_words, x='word', y='frequency', hover_data= ['word','frequency'] ,color='frequency',
#             title = 'İlan tanımlarında en çok kullanılan 25 kelime',height=400)
# fig.show()



Burada tüm ilan başlıkları tek bir listede toplanıyor ve daha sonra Tf-Idf ile her bir kelimenin dökümanın içerisindeki değeri belirleniyor. 


In [0]:
del(subset)
x_data = normalDistOfDataframe[['c0','c1','c2','description','title']]
#version1: ram bittiği için hem ilan başlığı hemde tanımı ile yapamadım. Yorum olarak bırakıyorum.
#titles = []
# descriptions = []
corpus = []
# for i in range(0,len(x_data)):
#   descriptions.append(x_data.iloc[i,3])
#   titles.append(x_data.iloc[i,4])
#   corpus.append(titles[i]+" "+ descriptions[i])

#verison2: sadece ilan başlığı ile yaptığım hali.
for i in range(0,len(x_data)):
  corpus.append(x_data.iloc[i,4])
  
del(x_data)

In [0]:
vec = tf().fit(corpus)
vec = vec.transform(corpus)
x = vec.toarray()
y = normalDistOfDataframe['price']
del(vec)

In [0]:

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30, random_state=39)
del(x)
del(y)
#print(len(x_train), len(x_test),len(y_train),len(y_test))
#y_train

In [13]:
randomForest = RandomForestRegressor(n_estimators=20)
randomForest.fit(x_train,y_train)
predicted = randomForest.predict(x_test)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predicted))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predicted))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicted)))

Mean Absolute Error: 18601.759239088486
Mean Squared Error: 646593211.2231296
Root Mean Squared Error: 25428.197168166083
